In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
import pandas as pd

In [ ]:
df_twitter = pd.read_csv('data/stock_tweets.csv')

In [ ]:
df_twitter

In [ ]:
#https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)

for tweet in df_twitter['Tweet']:
    text = tweet
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    positive = []
    neutral = []
    negative = []

    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = config.id2label[ranking[i]]
        s = scores[ranking[i]]

        if l == 'negative':
            negative.append(np.round(float(s), 4))
        elif l == 'neutral':
            neutral.append(np.round(float(s), 4))
        elif l == 'positive':
            positive.append(np.round(float(s), 4))



In [ ]:
print(len(positive))
print(len(negative))
print(len(neutral))